In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 2
GO = 1
vocab_size = 32000

In [6]:
train_Y = [i + [2] for i in train_Y]
test_Y = [i + [2] for i in test_Y]

In [7]:
from tensor2tensor.utils import beam_search

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, size_layer, num_layers, embedded_size, learning_rate):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.BasicRNNCell(size_layer,reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        embeddings = tf.Variable(tf.random_uniform([vocab_size, embedded_size], -1, 1))
        
        def forward(x, y, reuse = False):
            batch_size = tf.shape(x)[0]
            X_seq_len = tf.count_nonzero(x, 1, dtype = tf.int32)
            Y_seq_len = tf.count_nonzero(y, 1, dtype = tf.int32)
            with tf.variable_scope('model',reuse=reuse):
                encoder_embedded = tf.nn.embedding_lookup(embeddings, x)
                decoder_embedded = tf.nn.embedding_lookup(embeddings, y)
                rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
                last_output, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded,
                                                  sequence_length=X_seq_len,
                                                  dtype = tf.float32)
                
            with tf.variable_scope("decoder",reuse=reuse):
                
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = last_output)
                rnn_cells = tf.contrib.seq2seq.AttentionWrapper(
                    cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
                    attention_mechanism = attention_mechanism,
                    attention_layer_size = size_layer)
                
                initial_state = rnn_cells.zero_state(batch_size, tf.float32).clone(cell_state=last_state)
                outputs, _ = tf.nn.dynamic_rnn(rnn_cells, decoder_embedded, 
                                               sequence_length=Y_seq_len,
                                               initial_state = initial_state,
                                               dtype = tf.float32)
                
                return tf.layers.dense(outputs,vocab_size)
            
        main = tf.strided_slice(self.X, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        self.training_logits = forward(self.X, decoder_input, reuse = False)
        
        self.training_logits = self.training_logits[:, :tf.reduce_max(self.Y_seq_len)]
        self.training_logits = pad_second_dim(self.training_logits, tf.reduce_max(self.Y_seq_len))
            
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        initial_ids = tf.fill([batch_size], GO)
        def symbols_to_logits(ids):
            x = tf.contrib.seq2seq.tile_batch(self.X, 1)
            logits = forward(x, ids, reuse = True)
            return logits[:, tf.shape(ids)[1]-1, :]
        
        final_ids, final_probs, _ = beam_search.beam_search(
            symbols_to_logits,
            initial_ids,
            1,
            tf.reduce_max(self.X_seq_len),
            vocab_size,
            0.0,
            eos_id = EOS)
        
        self.fast_result = final_ids

In [8]:
size_layer = 512
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 128
epoch = 20

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (f

In [10]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [11]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[array([[[    1, 19508,  9278, 18378, 13421, 30227,  9547,  8103,  1436,
          11862,  9379, 30948, 23919,   473,  8926, 31710, 21259, 10352,
          31648, 27234, 28328, 24074, 15440,  5764, 16271, 23236, 27236,
           2995, 19841, 13161, 30411,   763, 19158,   958, 24045,  9018,
          22377]],
 
        [[    1, 12333,  1338, 23573,  5338, 16627, 12571,  1592, 25791,
          21522,  1206, 14757,  2262, 28976, 10605,  9821,  2487, 26057,
           7672, 30862, 17629, 14025,  3911, 12018, 18918,   365, 24643,
          29542, 19069, 20516, 26645,  8063, 16352, 11390,  2916,   469,
          28543]],
 
        [[    1, 20497, 10163, 30573, 24962,  1681,  7141, 31867, 25130,
          15475,  8273,  5719,   912, 30318, 18071, 20755, 17251, 17557,
          18977, 24487, 17478, 17010, 21247, 26161, 25048, 13653,  1981,
           7221, 20509,  2517, 22225, 30175,  8321, 13639, 29411, 23374,
          18908]],
 
        [[    1, 20467, 11620,  5707,    85, 21637,   688,  7

In [12]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 7.265123, training avg acc 0.099548
epoch 1, testing avg loss 6.825994, testing avg acc 0.119791


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 6.434877, training avg acc 0.134064
epoch 2, testing avg loss 6.269685, testing avg acc 0.141217


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 18, training avg loss 10.774070, training avg acc 0.063212
epoch 18, testing avg loss 10.603854, testing avg acc 0.064931


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 19, training avg loss 10.262059, training avg acc 0.064826
epoch 19, testing avg loss 10.124268, testing avg acc 0.060836


minibatch loop: 100%|██████████| 40/40 [00:05<00:00,  7.34it/s, accuracy=0.0538, cost=9.02]

epoch 20, training avg loss 9.853926, training avg acc 0.065893
epoch 20, testing avg loss 9.867109, testing avg acc 0.062831


In [13]:
from tensor2tensor.utils import bleu_hook

In [14]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)[:,0,:]
    result = []
    for row in p:
        result.append([i for i in row if i > 3])
    results.extend(result)

100%|██████████| 40/40 [02:18<00:00,  3.47s/it]


In [15]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 3])

In [16]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.00020161743